In [ ]:
Obtención de predicciones

podemos realizar peticiones a este utilizando el cliente de runtime de Amazon SageMaker mediante el uso del SDK para Python de AWS llamado boto3.

El código a continuación es completamente independiente y podría ser ejecutado desde cualquier otro servicio de AWS, como por ejemplo AWS Lambda o Amazon EC2. O desde la PC local, instalando previamente el SDK boto3.

In [ ]:
import boto3
import json

sagemaker_runtime = boto3.client('sagemaker-runtime')

request_body = json.dumps(test_data.values.tolist())

response = sagemaker_runtime.invoke_endpoint(EndpointName = endpoint_name,
                                             ContentType = 'application/json',
                                             Body = request_body)

predictions = json.loads(response['Body'].read())


Comparar predicción vs real

In [ ]:
def plot_confusion_matrix(test_set, predictions, classes, title):
    classes=np.array(classes)
    cm = confusion_matrix(test_set, predictions)
    
    fig, (ax1,ax2) = plt.subplots(1,2, gridspec_kw={'width_ratios': [2, 3]})
    im = ax1.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    ax1.figure.colorbar(im, ax=ax1)
    
    # We want to show all ticks...
    ax1.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title="Confusion Matrix",
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax1.get_yticklabels(), rotation=90, ha="center", rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax1.text(j, i, format(cm[i, j], 'd'),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    
    tn, fp, fn, tp = cm.ravel()
    accuracy = (tp+tn)/(tn+fp+fn+tp)
    precision = tp/(fp+tp)
    recall = tp/(fn+tp)
    specifity = tn/(tn+fp)
    
    ax2.axis('off')
    ax2.text(0,0.9, s='The overall model accuracy is {}% [ACCURACY]'.format(round(accuracy*100,2)), 
             size='12', ha='left', va='center')
    
    ax2.text(0,0.7, s='Out of the customers the model predicted as will churn, {}% will actually churn [PRECISION]'.format(round(precision*100,2)), 
             size='12', ha='left', va='center')
    
    ax2.text(0,0.5, s='The model will catch {}% of the customers who will actually churn [RECALL / SENSITIVITY]'.format(round(recall*100,2)), 
             size='12', ha='left', va='center')
    
    ax2.text(0,0.3, s='The model will catch {}% of the customers who will actually NOT churn [SPECIFITY]'.format(round(specifity*100,2)), 
             size='12', ha='left', va='center')
    
    fig.set_figheight(3)
    fig.set_figwidth(16)
    fig.suptitle(title, fontsize=16)
    plt.show()


Antes de poder graficar la matriz de confusión, debemos cambiar las predicciones obtenidas las cuales representan el porcentaje de probabilidad de hacer Churn, por un 0 o un 1 utilizando el umbral obtenido previamente.

In [ ]:
decision_threshold = best_model_found['Threshold']
predictions=[1 if prediction >= decision_threshold else 0 for prediction in predictions]


In [ ]:
labels = ['Not Churn','Churn']
plot_confusion_matrix(test_target, predictions, labels, 'Best Model')
